# 1. Uploading the data

## 1.1. The train, val, and test data

- We uploaded `THEMIS_DAYIR_TT_MergedMosaic.tif` directly to Earth Engine as no pre-processing steps were required, with EE path `projects/esg-satelite/assets/mars/features/themis_v1/pre/themis`.
    - This image covers all areas of Tempe Terra where fault lines were drawn by hand.
    - It is in CRS EPSG:3857.